**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [3]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_crispri_hcrff")
dir(fdiry)

[1] "peak.annotation.crispri_casa_peak.bed.gz"

In [4]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_crispri_hcrff")
dir(fdiry)

[1] "crispri_casa_peak.bed"                
[2] "crispri_casa_peak.tsv"                
[3] "crispri_gene_tss.tsv"                 
[4] "description.tsv"                      
[5] "K562_HCRFF_20genes_TSS_TES_coords.tsv"

## Import data

In [10]:
folder = "annotation_crispri_hcrff"
fdiry  = file.path(FD_RES, "results", "region", folder)
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath)

dat_cnames = dat
print(dim(dat))
dat

Rows: 8 Columns: 2
── Column specification ─────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): Name, Description

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 8 2


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start position
End,End position
Name,Name
Score,Score
Strand,[+-.]; Use '.' if no strand is assigned.
Gene_Symbol,Gene symbol; Gene that is screened for CRISPRi-FlowFish
Gene_ENS,Gene Ensembl ID; Gene that is screened for CRISPRi-FlowFish


In [12]:
### init
annotation = "CRISPRi_HCRFF"
label      = "CASA"
cnames = dat_cnames$Name
cnames = c("Chrom_ATAC", "Start_ATAC", "End_ATAC", cnames, "Overlap")

###
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    folder)
fname = "peak.annotation.crispri_casa_peak.bed.gz"
fpath = file.path(fdiry, fname)

### read data
dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
dat = dat %>% dplyr::mutate(
    Peak_ATAC  = paste(Chrom_ATAC, Start_ATAC, End_ATAC, sep="_"),
    Annotation = annotation,
    Label      = label
)

### assign and show
dat_peak_annot = dat
print(dim(dat))
head(dat)

[1] 87 15


Chrom_ATAC,Start_ATAC,End_ATAC,Chrom,Start,End,Name,Score,Strand,Gene_Symbol,Gene_ENS,Overlap,Peak_ATAC,Annotation,Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
chr11,5249943,5250880,chr11,5249847,5250847,chr11:5249847-5250847,1.890890,.,HBG1,ENST00000330597.5,904,chr11_5249943_5250880,CRISPRi_HCRFF,CASA
chr11,5249943,5250880,chr11,5249847,5250847,chr11:5249847-5250847,1.922956,.,HBG2,ENST00000336906.6,904,chr11_5249943_5250880,CRISPRi_HCRFF,CASA
chr11,5254190,5254453,chr11,5253647,5254647,chr11:5253647-5254647,1.504404,.,HBG1,ENST00000330597.5,263,chr11_5254190_5254453,CRISPRi_HCRFF,CASA
chr11,5254190,5254453,chr11,5253647,5254647,chr11:5253647-5254647,1.430535,.,HBG2,ENST00000336906.6,263,chr11_5254190_5254453,CRISPRi_HCRFF,CASA
chr11,5269440,5270695,chr11,5269147,5271147,chr11:5269147-5271147,1.906478,.,HBE1,ENST00000336906.6,1255,chr11_5269440_5270695,CRISPRi_HCRFF,CASA
chr11,5275424,5276195,chr11,5275847,5276247,chr11:5275847-5276247,1.173871,.,HBG1,ENST00000330597.5,348,chr11_5275424_5276195,CRISPRi_HCRFF,CASA


## Arrange and summarize

In [13]:
dat = dat_peak_annot
dat = dat %>% 
    dplyr::group_by(Peak_ATAC, Annotation, Label) %>%
    dplyr::summarise(
        Count = n(), 
        Note  = paste(Gene_Symbol, collapse="|"),
        .groups = "drop")

dat = dat %>% dplyr::rename("Peak" = "Peak_ATAC")

### assign and show
dat_peak_annot_count = dat
print(dim(dat))
head(dat)

[1] 57  5


Peak,Annotation,Label,Count,Note
<chr>,<chr>,<chr>,<int>,<chr>
chr11_33868682_33871379,CRISPRi_HCRFF,CASA,1,LMO2
chr11_33880631_33881416,CRISPRi_HCRFF,CASA,1,LMO2
chr11_33881831_33882405,CRISPRi_HCRFF,CASA,1,LMO2
chr11_33936760_33937819,CRISPRi_HCRFF,CASA,1,LMO2
chr11_33941500_33942221,CRISPRi_HCRFF,CASA,1,LMO2
chr11_33944510_33945297,CRISPRi_HCRFF,CASA,3,CAPRIN1|CAT|LMO2


In [14]:
dat = dat_peak_annot_count
table(dat$Count)


 1  2  3 
38  8 11 

## Save results

In [15]:
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "summary")
fname = "peak.summary.crispri_hcrff_casa.tsv"
fpath = file.path(fdiry, fname)

dat = dat_peak_annot_count
write_tsv(dat, fpath)

In [3]:
### set file path
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input")
fname = "peak.annotation.crispri_casa_peak.bed.gz"
fpath = file.path(fdiry, fname)


### import data
cnames = c(
    "Chrom", "Start", "End",
    "Chrom_CASA", "Start_CASA", "End_CASA", 
    "Name", "Score", "Strand", "Gene_Symbol", "Gene_ENS", 
    "Overlap")
dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
dat = dat %>% mutate(
    Peak = paste(Chrom, Start, End, sep="_"),
    CASA = paste(Chrom_CASA, Start_CASA, End_CASA, sep="_"),
)

### assign
dat_peak_annot_crispri = dat
print(dim(dat))
head(dat)

[1] 87 14


Chrom,Start,End,Chrom_CASA,Start_CASA,End_CASA,Name,Score,Strand,Gene_Symbol,Gene_ENS,Overlap,Peak,CASA
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
chr11,5249943,5250880,chr11,5249847,5250847,chr11:5249847-5250847,1.890890,.,HBG1,ENST00000330597.5,904,chr11_5249943_5250880,chr11_5249847_5250847
chr11,5249943,5250880,chr11,5249847,5250847,chr11:5249847-5250847,1.922956,.,HBG2,ENST00000336906.6,904,chr11_5249943_5250880,chr11_5249847_5250847
chr11,5254190,5254453,chr11,5253647,5254647,chr11:5253647-5254647,1.504404,.,HBG1,ENST00000330597.5,263,chr11_5254190_5254453,chr11_5253647_5254647
chr11,5254190,5254453,chr11,5253647,5254647,chr11:5253647-5254647,1.430535,.,HBG2,ENST00000336906.6,263,chr11_5254190_5254453,chr11_5253647_5254647
chr11,5269440,5270695,chr11,5269147,5271147,chr11:5269147-5271147,1.906478,.,HBE1,ENST00000336906.6,1255,chr11_5269440_5270695,chr11_5269147_5271147
chr11,5275424,5276195,chr11,5275847,5276247,chr11:5275847-5276247,1.173871,.,HBG1,ENST00000330597.5,348,chr11_5275424_5276195,chr11_5275847_5276247


## Arrange

In [5]:
###
dat = dat_peak_annot_crispri
dat = dat %>% 
    dplyr::select(Peak, CASA, Score, Gene_Symbol, Gene_ENS, Overlap)

###
dat_peak_summary_crispri = dat
print(dim(dat))
head(dat)

[1] 87  6


Peak,CASA,Score,Gene_Symbol,Gene_ENS,Overlap
<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
chr11_5249943_5250880,chr11_5249847_5250847,1.890890,HBG1,ENST00000330597.5,904
chr11_5249943_5250880,chr11_5249847_5250847,1.922956,HBG2,ENST00000336906.6,904
chr11_5254190_5254453,chr11_5253647_5254647,1.504404,HBG1,ENST00000330597.5,263
chr11_5254190_5254453,chr11_5253647_5254647,1.430535,HBG2,ENST00000336906.6,263
chr11_5269440_5270695,chr11_5269147_5271147,1.906478,HBE1,ENST00000336906.6,1255
chr11_5275424_5276195,chr11_5275847_5276247,1.173871,HBG1,ENST00000330597.5,348


## Save tables

In [6]:
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input")
fname = "peak.summary.crispri_casa_peak.tsv"
fpath = file.path(fdiry, fname)

write_tsv(dat_peak_summary_crispri, fpath)